In [16]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from pymodbus.client import ModbusTcpClient
import time

In [17]:
cam = 0
coil_address = 1536 #ModBus Y0
PLC_IP = "192.168.1.111"  #PLC's IP address
PLC_PORT = 502           # Default Modbus TCP port

model_version = "my_model_14.0.keras"
Im_Width = 128
Im_Height = 128

In [ ]:
def modbus(PLC_IP, PLC_PORT):

    # Create Modbus client
    client = ModbusTcpClient(PLC_IP, port=PLC_PORT)

    # Connect to the PLC
    if client.connect():
        print("Connection successful")
    else:
        print("Failed to connect to PLC")

In [ ]:
def PLC(coil_address,value):
    write_result = client.write_coil(address=coil_address, value=value)  # Write 'True' to turn ON
    if write_result.isError():
        print("Write Error:", write_result)
    else:
        print(f"Output Y0 turned ON (Coil {coil_address})")

In [18]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [19]:
def measure_brightness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [20]:
def measure_edge_richness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    edges = cv2.Canny(gray, 200, 400)  # Apply Canny edge detection
    edge_richness = np.sum(edges)  # Sum of edge pixel intensities
    return edge_richness

In [21]:
def select_best_frame(frame_buffer):

    best_frame = None
    highest_brightness = 150
    highest_edge_richness = 100000

    for frame in frame_buffer:

        #Crop
        cropped_frame = crop_image(frame, 220, 0, 200, 200)

        # Measure brightness
        brightness = measure_brightness(cropped_frame)

        if brightness > 150:
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame
    
        elif brightness > 50 and brightness < 100:
            # Apply edge detection
            edge_richness = measure_edge_richness(cropped_frame)
            # Update the best frame if this one is better
            if edge_richness > highest_edge_richness:
                highest_edge_richness = edge_richness
                best_frame = frame  # Save the cropped frame as the best frame

    return best_frame  # Returns None if no frames meet the threshold


In [22]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [23]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

   # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        PLC(coil_address,False)
        return "Non-Defective", prediction       
        
    else:
        PLC(coil_address,True)
        return "Defective", prediction
    


In [26]:
def capture(cam, model_path, target_size=(Im_Width, Im_Height)):

    cap = cv2.VideoCapture(cam)  # Open webcam
    frame_buffer = []
    max_frames = 10  # Number of frames to evaluate

    # Load the model
    print("Loading model...") 
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:

        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        frame_buffer.append(frame)

        # Process the buffer when it's full
        if len(frame_buffer) == max_frames:
            best_frame = select_best_frame(frame_buffer)
            frame_buffer = []  # Clear the buffer

            # Process and predict
            if best_frame is not None:
                feedback, confidence = predict_frame(best_frame, model, target_size)
                print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

                prob = abs(round(confidence,2)- 0.5)*200

                # Display feedback on the frame
                cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(best_frame, f"Confidence: {prob:.0f}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Display the best frame
                cv2.imshow("Best Frame", best_frame)

            else:
                # Load and display the "no-bottle" image in the same window
                image_path = "no-bottle.bmp"
                image = cv2.imread(image_path)

                if image is None:
                    print("Error: Could not load 'no-bottle' image.")
                else:
                    cv2.putText(image, "No Bottle!", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.imshow("Best Frame", image)

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


In [27]:
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    modbus(PLC_IP, PLC_PORT)
    capture(cam, model_path, target_size=(Im_Width, Im_Height))

Loading model...
Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Raw Prediction: 0.001657717628404498
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Raw Prediction: 0.0002801687514875084
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Raw Prediction: 0.0002086734602926299
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Raw Prediction: 0.00016693158249836415
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Raw Prediction: 0.00012788483581971377
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Raw Prediction: 0.0001274791720788926
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Raw Prediction: 0.00018719941726885736
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Raw Prediction: 0.0002032129414146766
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━